In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [5]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [6]:
# 计算区域高程
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
minmax = elevation.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(minmax.getInfo())

{'elevation_max': 5896, 'elevation_min': 4753}


In [ ]:
region = 'tianjin'

## 永久性水体样本和非水体样本，抽样
pw_nw = ee.FeatureCollection('users/311605001111/' + region + '_kmean100_noseason2016')
permanent_water = pw_nw.filter(ee.Filter.eq('waterclass',1)).randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.4))
no_water = pw_nw.filter(ee.Filter.eq('waterclass',0)).randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.2))
print('抽样的永久性水体： ',permanent_water.size().getInfo())
print('抽样的非水体： ',no_water.size().getInfo())

## 季节性样本，抽样
sw = ee.FeatureCollection('users/311605001111/' + region + '_kmean100_season2016').randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.4))
print('抽样的季节性水体： ',sw.size().getInfo())

## 季节性样本，分类
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
cluster_samples = ee.FeatureCollection([permanent_water,no_water]).flatten()
clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
q = ee.FeatureCollection(sw.cluster(clusterer))
q1 = q.filter(ee.Filter.eq('cluster',1))
q0 = q.filter(ee.Filter.eq('cluster',0))
b1 = ee.FeatureCollection(q1).filter(ee.Filter.lt('AWEI',0)).size()
b2 = ee.FeatureCollection(q1).filter(ee.Filter.gt('AWEI',0)).size()
c1 = ee.FeatureCollection(q0).filter(ee.Filter.lt('AWEI',0)).size()
c2 = ee.FeatureCollection(q0).filter(ee.Filter.gt('AWEI',0)).size()

qq1 = ee.Algorithms.If(ee.Number(b1).lt(ee.Number(b2)),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
qq0 = ee.Algorithms.If(ee.Number(c1).lt(ee.Number(c2)),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
seasonWater = ee.FeatureCollection([qq1,qq0]).flatten()

## 合并
total_Samples = ee.FeatureCollection([permanent_water,no_water,seasonWater]).flatten()
print("筛选前，总的样本数： ",total_Samples.size().getInfo())

## 异常值去除
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
#### cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
#### k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

filtered_samples = k_mean(total_Samples)
print("筛选后，剩余的样本数： ",filtered_samples.size().getInfo())

print('no-water')
print(filtered_samples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.lt('AWEI',0)).size().getInfo())
print(filtered_samples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.gt('AWEI',0)).size().getInfo())
print('water')
print(filtered_samples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.lt('AWEI',0)).size().getInfo())
print(filtered_samples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.gt('AWEI',0)).size().getInfo())

# 比较

## 约束

In [ ]:
region = 'prd'

## 永久性水体样本和非水体样本，抽样
pw_nw = ee.FeatureCollection('users/311605001111/' + region + '_kmean100_noseason2016')
permanent_water = pw_nw.filter(ee.Filter.eq('waterclass',1)).randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.4))
no_water = pw_nw.filter(ee.Filter.eq('waterclass',0)).randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.2))
print('抽样的永久性水体： ',permanent_water.size().getInfo())
print('抽样的非水体： ',no_water.size().getInfo())

## 季节性样本，抽样
sw = ee.FeatureCollection('users/311605001111/' + region + '_kmean100_season2016').randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.4))
print('抽样的季节性水体： ',sw.size().getInfo())

## 季节性样本，分类
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
cluster_samples = ee.FeatureCollection([permanent_water,no_water]).flatten()
clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
q = ee.FeatureCollection(sw.cluster(clusterer))
q1 = q.filter(ee.Filter.eq('cluster',1))
q0 = q.filter(ee.Filter.eq('cluster',0))
b1 = ee.FeatureCollection(q1).filter(ee.Filter.lt('AWEI',0)).size()
b2 = ee.FeatureCollection(q1).filter(ee.Filter.gt('AWEI',0)).size()
c1 = ee.FeatureCollection(q0).filter(ee.Filter.lt('AWEI',0)).size()
c2 = ee.FeatureCollection(q0).filter(ee.Filter.gt('AWEI',0)).size()

qq1 = ee.Algorithms.If(ee.Number(b1).lt(ee.Number(b2)),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
qq0 = ee.Algorithms.If(ee.Number(c1).lt(ee.Number(c2)),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
seasonWater1 = ee.FeatureCollection([qq1,qq0]).flatten()


m1 = seasonWater1.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.lt('AWEI',0)).size()
print(m1.getInfo())
m2 = seasonWater1.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.gt('AWEI',0)).size()
print(m2.getInfo())

n1 = seasonWater1.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.lt('AWEI',0)).size()
print(n1.getInfo())
n2 = seasonWater1.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.gt('AWEI',0)).size()
print(n2.getInfo())

## 绘图

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

x = 'NDVI'
y = 'mNDWI'
z = 'AWEI'
filtered_samples = seasonWater1
land_x =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',0)).aggregate_array(x).getInfo()) 
land_y =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',0)).aggregate_array(y).getInfo()) 
land_z =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',0)).aggregate_array(z).getInfo()) 
water_x =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',1)).aggregate_array(x).getInfo()) 
water_y =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',1)).aggregate_array(y).getInfo()) 
water_z =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',1)).aggregate_array(z).getInfo()) 

# 创建一个三维的绘图工程
plt.figure(figsize=(10, 8))

ax = plt.subplot(111, projection='3d')  
# 绘制数据点
ax.scatter(water_x, water_y, water_z, s=2, c='blue', label = 'water')
ax.scatter(land_x, land_y, land_z, s=2,c='red', label = 'land')  

ax.view_init(32,-51)
# 坐标轴
ax.set_xlim3d(-0.85,1.1)
ax.set_ylim3d(-0.8,1.1)
ax.set_zlim3d(-15000,5300)
ax.set_xlabel(x,fontsize = 11)
ax.set_ylabel(y,fontsize = 11)
ax.set_zlabel(z,labelpad = 11,rotation=90,fontsize = 11)
plt.xticks([-0.5,0,0.5,1.0])
plt.yticks([-0.5,0,0.5,1.0])
plt.title('Title: NDVI/MNDWI/AWEI') 
plt.legend(loc='upper right',markerscale=4,prop={'size':12})
plt.show()

In [ ]:
x = 'NDWI'
y = 'mNDWI'
z = 'AWEI'
filtered_samples = seasonWater1
land_x =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',0)).aggregate_array(x).getInfo()) 
land_y =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',0)).aggregate_array(y).getInfo()) 
land_z =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',0)).aggregate_array(z).getInfo()) 
water_x =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',1)).aggregate_array(x).getInfo()) 
water_y =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',1)).aggregate_array(y).getInfo()) 
water_z =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',1)).aggregate_array(z).getInfo()) 

plt.figure(figsize=(8, 6))
plt.scatter(water_x,water_y,s=5,c='blue', label = 'season_water')
plt.scatter(land_x,land_y,s=5,c='red', label = 'season_land')

# B3/B6
plt.xlim(-1,1)
plt.ylim(-1,1)
plt.tick_params(labelsize=18)

plt.xlabel(x,fontsize=25)    
plt.ylabel(y,fontsize=25)

plt.title('B3/B6',fontsize=27)
plt.legend(prop={'size':18},markerscale=6,loc = 'upper right')
# plt.legend(prop={'size':11},markerscale=3,loc = 'lower right')
plt.show()

## 不约束

In [ ]:
region = 'prd'

## 季节性样本，抽样
sw = ee.FeatureCollection('users/311605001111/' + region + '_kmean100_season2016').randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.4))
print('抽样的季节性水体： ',sw.size().getInfo())

## 季节性样本，分类
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
clusterer = ee.Clusterer.wekaKMeans(2).train(sw,bands)
q = ee.FeatureCollection(sw.cluster(clusterer))
q1 = q.filter(ee.Filter.eq('cluster',1))
q0 = q.filter(ee.Filter.eq('cluster',0))
b1 = ee.FeatureCollection(q1).filter(ee.Filter.lt('AWEI',0)).size()
b2 = ee.FeatureCollection(q1).filter(ee.Filter.gt('AWEI',0)).size()
c1 = ee.FeatureCollection(q0).filter(ee.Filter.lt('AWEI',0)).size()
c2 = ee.FeatureCollection(q0).filter(ee.Filter.gt('AWEI',0)).size()

qq1 = ee.Algorithms.If(ee.Number(b1).lt(ee.Number(b2)),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
qq0 = ee.Algorithms.If(ee.Number(c1).lt(ee.Number(c2)),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
seasonWater = ee.FeatureCollection([qq1,qq0]).flatten()

m1 = ee.FeatureCollection(qq0).filter(ee.Filter.lt('AWEI',0)).size()
print(m1.getInfo())
m2 = ee.FeatureCollection(qq0).filter(ee.Filter.gt('AWEI',0)).size()
print(m2.getInfo())

n1 = ee.FeatureCollection(qq1).filter(ee.Filter.lt('AWEI',0)).size()
print(n1.getInfo())
n2 = ee.FeatureCollection(qq1).filter(ee.Filter.gt('AWEI',0)).size()
print(n2.getInfo())

## 绘图

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

x = 'NDVI'
y = 'mNDWI'
z = 'AWEI'
filtered_samples = seasonWater
land_x =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',0)).aggregate_array(x).getInfo()) 
land_y =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',0)).aggregate_array(y).getInfo()) 
land_z =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',0)).aggregate_array(z).getInfo()) 
water_x =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',1)).aggregate_array(x).getInfo()) 
water_y =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',1)).aggregate_array(y).getInfo()) 
water_z =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',1)).aggregate_array(z).getInfo()) 

# 创建一个三维的绘图工程
plt.figure(figsize=(10, 8))

ax = plt.subplot(111, projection='3d')  
# 绘制数据点
ax.scatter(water_x, water_y, water_z, s=2, c='blue', label = 'remain_water')
ax.scatter(land_x, land_y, land_z, s=2,c='red', label = 'remain_land')  

ax.view_init(32,-51)
# 坐标轴
ax.set_xlim3d(-0.85,1.1)
ax.set_ylim3d(-0.8,1.1)
ax.set_zlim3d(-15000,5300)
ax.set_xlabel(x,fontsize = 11)
ax.set_ylabel(y,fontsize = 11)
ax.set_zlabel(z,labelpad = 11,rotation=90,fontsize = 11)
plt.xticks([-0.5,0,0.5,1.0])
plt.yticks([-0.5,0,0.5,1.0])
plt.title('Title: NDVI/MNDWI/AWEI') 
plt.legend(loc='upper right',markerscale=4,prop={'size':12})
plt.show()

In [ ]:
x = 'NDWI'
y = 'mNDWI'
z = 'AWEI'
filtered_samples = seasonWater
land_x =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',0)).aggregate_array(x).getInfo()) 
land_y =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',0)).aggregate_array(y).getInfo()) 
land_z =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',0)).aggregate_array(z).getInfo()) 
water_x =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',1)).aggregate_array(x).getInfo()) 
water_y =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',1)).aggregate_array(y).getInfo()) 
water_z =np.array(filtered_samples.filter(ee.Filter.eq('waterclass',1)).aggregate_array(z).getInfo()) 

plt.figure(figsize=(8, 6))
plt.scatter(water_x,water_y,s=5,c='blue', label = 'season_water')
plt.scatter(land_x,land_y,s=5,c='red', label = 'season_land')

# B3/B6
plt.xlim(-1,1)
plt.ylim(-1,1)
plt.tick_params(labelsize=18)

plt.xlabel(x,fontsize=25)    
plt.ylabel(y,fontsize=25)

plt.title('B3/B6',fontsize=27)
plt.legend(prop={'size':18},markerscale=6,loc = 'upper right')
# plt.legend(prop={'size':11},markerscale=3,loc = 'lower right')
plt.show()